In [1]:
# First let's do an import. If you get an Import Error, double check that your Kernel is correct..

from dotenv import load_dotenv
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [2]:
load_dotenv(override=True)

True

In [3]:
import os
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:8]}")
else:
    print("Anthropic API Key not set - please head to the troubleshooting guide in the setup folder")

Anthropic API Key exists and begins sk-ant-a


In [4]:
claude = Anthropic()

In [5]:
messages = [{"role": "user", "content": "say hello"}]

In [6]:
# Control sampling with temperature and top_p
# Lower temperature = more deterministic; higher = more creative
# top_p (nucleus sampling) limits sampling to the most probable tokens mass.

model_name = "claude-3-7-sonnet-latest"


temperature = 0.1  # 0.0-1.0 typical
top_p = 1.0        # 0.0-1.0

response = claude.messages.create(
    model=model_name,
    messages=messages,
    max_tokens=1000,
    temperature=temperature,
    top_p=top_p,
)

answer = response.content[0].text
display(Markdown(answer))


Hello! How can I assist you today?

In [10]:
# Vision: system + user prompts with multiple images
from base64 import b64encode
from pathlib import Path

# --- Configure prompts ---
system_prompt = (
"""
You are an e-scooter parking enforcement officer. Apply the policy fairly and pragmatically.

Key principle: PASS the scooter if it meets AT LEAST ONE of the acceptable parking criteria. Only FAIL if it clearly violates safety rules or blocks access.

Provide your decision as PASS or FAIL with a concise reason.

Your reasoning must reference concrete visual evidence. Be concise but specific about what you observe.

"""
)

user_prompt = (
"""
The e-scooter must be fully visible in the photo, be standing upright and not be held by the rider.

ANALYSIS PROCESS:
For options 1-3 (designated parking areas):
1. First, scan the entire ground area for ALL visible markings, painted lines, mats, or structures
2. Identify what defines the BOUNDARY/EDGE of any parking area (this is where the parking space begins and ends)
3. Then identify any SYMBOLS or markings inside that boundary
4. Check if the entire scooter (wheels, deck, body) is within the parking area boundary

For option 4 (non-designated areas):
A. Identify the primary purpose of the area (pedestrian walkway, road shoulder, parking lot, driveway, etc.)
B. Assess if the scooter leaves sufficient space for the primary users of that area
C. Check if the scooter is positioned to minimize potential hazards (visible, away from high-traffic zones)
D. Determine if the placement creates an obstruction that would:
   - Force pedestrians to walk around it on a sidewalk
   - Prevent vehicles from using the space as intended
   - Create a safety hazard due to reduced visibility or access
E. If the scooter leaves adequate clearance for intended users and doesn't create hazards, it passes criterion 4

The scooter must be parked in ONE of these locations:

1. A designated micromobility parking space with metal rails/barriers around it, with boards displaying colorful illustrations attached to the rails, and optionally the "P" symbol and/or words like "e-scooter corral", OR

2. A parking area defined by a white, thick outline/border (rectangular) painted on the ground. This area may optionally contain symbols inside such as a white e-scooter icon, "P" symbol or bicycle painted on it. The e-scooter must be parked with its entire body fully inside this white-outlined boundary, OR

3. A parking area consisting of a dark-colored mat or painted surface (black, dark gray, or dark blue). This area may optionally contain symbols inside such as a white e-scooter icon, "P" symbol or bicycle painted on it. The e-scooter must be parked with its entire body fully inside this dark-colored area, OR

4. On a sidewalk, walkway, or paved roadway area (including shoulders, parking lanes, driveways, or access roads) where the scooter:
   - Does not obstruct pedestrian traffic or wheelchair users
   - Does not block building entrances/exits
   - Does not block curb cuts or wheelchair ramps
   - Does not block transit stops or bus shelters
   - Does not impede vehicle traffic flow
   - Is positioned to minimize hazards for all passersby
   - Leaves sufficient clear space for intended users of that area


KEY DISTINCTIONS:
- BOUNDARY = the outline, edge, or physical border that defines where the parking space is
- SYMBOLS = icons, letters, or shapes inside the parking area (informational, optional in some cases)
- "Fully inside" = the entire scooter including wheels must be within the parking area boundary
- Partial/incomplete white lines are acceptable if at least 2-3 sides or corner markers are visible that define a rectangular area
- Faded markings that are still discernible count as valid boundaries


"""
)

# --- Provide images from local folder ---
# Reads all jpg/jpeg/png files from ./calgary_images and attaches as base64
images_dir = Path("calgary_images")
local_paths = sorted([
    p for p in images_dir.glob("**/*")
    if p.suffix.lower() in {".jpg", ".jpeg", ".png"}
])

content = [{"type": "text", "text": user_prompt}]

for p in local_paths:
    data = p.read_bytes()
    suffix = p.suffix.lower()
    
    # Detect actual file type from content, not just extension
    if data[:2] == b'\xff\xd8':
        media_type = "image/jpeg"
    elif data[:8] == b'\x89PNG\r\n\x1a\n':
        media_type = "image/png"
    elif suffix in {".jpg", ".jpeg"}:
        media_type = "image/jpeg"
    else:
        media_type = "image/png"
    
    content.append({
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": media_type,
            "data": b64encode(data).decode("utf-8"),
        },
    })

# --- Sampling controls ---
temperature = 0.1
top_p = 1.0
max_tokens = 1000

response = claude.messages.create(
    model=model_name,
    system=system_prompt,
    messages=[{"role": "user", "content": content}],
    temperature=temperature,
    top_p=top_p,
    max_tokens=max_tokens,
)

answer = response.content[0].text
display(Markdown(answer))


PASS - The scooter is parked on a sidewalk with sufficient space for pedestrians to pass by. It is standing upright and positioned in a way that doesn't obstruct pedestrian traffic. The scooter is placed at the edge of the sidewalk, leaving the main walkway clear for pedestrians. This meets criterion 4 for acceptable parking in non-designated areas, as it doesn't create a hazard or obstruction for sidewalk users.

In [11]:
# Batch-evaluate local images and show a results table
import json
import re
from base64 import b64encode
from pathlib import Path
import pandas as pd

# Collect images]
images_dir = Path("calgary_images")
local_paths = sorted([
    p for p in images_dir.glob("**/*")
    if p.suffix.lower() in {".jpg", ".jpeg", ".png"}
])

rows = []
for p in local_paths:
    data = p.read_bytes()
    suffix = p.suffix.lower()
    
    # Detect actual file type from content
    if data[:2] == b'\xff\xd8':
        media_type = "image/jpeg"
    elif data[:8] == b'\x89PNG\r\n\x1a\n':
        media_type = "image/png"
    elif suffix in {".jpg", ".jpeg"}:
        media_type = "image/jpeg"
    else:
        media_type = "image/png"

    content = [
        {"type": "text", "text": user_prompt},
        {"type": "text", "text": f"Filename: {p.name}"},
        {"type": "image", "source": {"type": "base64", "media_type": media_type, "data": b64encode(data).decode("utf-8")}},
    ]

    resp = claude.messages.create(
        model=model_name,
        system=system_prompt,
        messages=[{"role": "user", "content": content}],
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
    )

    text = resp.content[0].text

    # Parse strict JSON if present
    parsed = None
    try:
        parsed = json.loads(text)
    except Exception:
        match = re.search(r"\{[\s\S]*\}\s*$", text)
        if match:
            try:
                parsed = json.loads(match.group(0))
            except Exception:
                parsed = None

    decision = None
    reason = None
    confidence = None
    if isinstance(parsed, dict):
        decision = parsed.get("decision")
        reason = parsed.get("reason")
        confidence = parsed.get("confidence")

    if decision is None:
        decision = "UNKNOWN"
    if reason is None:
        reason = text.strip()

    # Expected label from filename (e.g., contains 'pass' or 'fail')
    name_lower = p.name.lower()
    expected = None
    if "pass" in name_lower:
        expected = "PASS"
    elif "fail" in name_lower:
        expected = "FAIL"

    # Actual label inferred from the reason text (look for the words PASS/FAIL)
    reason_upper = reason.upper()
    actual_from_reason = None
    if "PASS" in reason_upper and "FAIL" not in reason_upper:
        actual_from_reason = "PASS"
    elif "FAIL" in reason_upper and "PASS" not in reason_upper:
        actual_from_reason = "FAIL"

    matches_reason = (expected == actual_from_reason) if expected is not None and actual_from_reason is not None else None

    rows.append({
        "image": p.name,
        "expected": expected,
        "decision": decision,
        "actual_from_reason": actual_from_reason,
        "matches_reason": matches_reason,
        "confidence": confidence,
        "reason": reason,
    })

results_df = pd.DataFrame(rows)
# Preferred column order
col_order = ["image", "expected", "decision", "actual_from_reason", "matches_reason", "confidence", "reason"]
results_df = results_df[col_order]
display(results_df)

# Optional: save to CSV
results_df.to_csv("calgary_results.csv", index=False)
print("Saved table to calgary_results.csv")

,image,expected,decision,actual_from_reason,matches_reason,confidence,reason
0,Calgary_v6_model_should_fail.png,FAIL,UNKNOWN,FAIL,True,None,FAIL - The image only shows a partial view of ...
1,Calgary_v6_model_should_fail_2.jpg,FAIL,UNKNOWN,None,None,None,FAIL - The e-scooter is parked on a sidewalk i...
2,calagry_sidewalk_should_pass_9.jpg,PASS,UNKNOWN,PASS,True,None,PASS - The e-scooter is parked upright on a wi...
3,calgary_sidewalk_should_fail.png,FAIL,UNKNOWN,FAIL,True,None,FAIL\n\nThe scooter is parked in the middle of...
4,calgary_sidewalk_should_narrow_1.jpg,None,UNKNOWN,PASS,None,None,PASS - The scooter is parked on what appears t...
5,calgary_sidewalk_should_narrow_2.jpg,None,UNKNOWN,FAIL,None,None,FAIL - The image only shows the handlebars/upp...
6,calgary_sidewalk_should_narrow_3.jpg,None,UNKNOWN,FAIL,None,None,FAIL - The scooter is parked on what appears t...
7,calgary_sidewalk_should_pass_1.jpg,PASS,UNKNOWN,PASS,True,None,PASS - The scooter is parked on a sidewalk wit...
8,calgary_sidewalk_should_pass_2.jpg,PASS,UNKNOWN,PASS,True,None,PASS - The scooter is parked upright on a side...
9,calgary_sidewalk_should_pass_3.png,PASS,UNKNOWN,PASS,True,None,PASS\n\nThe scooter is parked on what appears ...


Saved table to calgary_results.csv


In [9]:
# Analyze cases where expected ≠ actual result
import json
import re
from base64 import b64encode
from pathlib import Path
import pandas as pd

# Collect images from calgary_images
images_dir = Path("calgary_images")
local_paths = sorted([
    p for p in images_dir.glob("**/*")
    if p.suffix.lower() in {".jpg", ".jpeg", ".png"}
])

print(f"Found {len(local_paths)} images")

# Process all images to find mismatches
rows = []
for p in local_paths:
    print(f"Processing {p.name}...")
    
    # Determine expected result from filename
    name_lower = p.name.lower()
    expected = None
    if "should_pass" in name_lower or "pass" in name_lower:
        expected = "PASS"
    elif "should_fail" in name_lower or "fail" in name_lower:
        expected = "FAIL"
    
    data = p.read_bytes()
    suffix = p.suffix.lower()
    
    # Detect actual file type from content
    if data[:2] == b'\xff\xd8':
        media_type = "image/jpeg"
    elif data[:8] == b'\x89PNG\r\n\x1a\n':
        media_type = "image/png"
    elif suffix in {".jpg", ".jpeg"}:
        media_type = "image/jpeg"
    else:
        media_type = "image/png"

    content = [
        {"type": "text", "text": user_prompt},
        {"type": "text", "text": f"Filename: {p.name}"},
        {"type": "image", "source": {"type": "base64", "media_type": media_type, "data": b64encode(data).decode("utf-8")}},
    ]

    resp = claude.messages.create(
        model=model_name,
        system=system_prompt,
        messages=[{"role": "user", "content": content}],
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
    )

    text = resp.content[0].text
    print(f"Raw response: {text}")
    
    # Parse JSON if present
    parsed = None
    try:
        parsed = json.loads(text)
    except Exception:
        match = re.search(r"\{[\s\S]*\}\s*$", text)
        if match:
            try:
                parsed = json.loads(match.group(0))
            except Exception:
                parsed = None

    decision = None
    reason = None
    confidence = None
    if isinstance(parsed, dict):
        decision = parsed.get("decision")
        reason = parsed.get("reason")
        confidence = parsed.get("confidence")

    if decision is None:
        decision = "UNKNOWN"
    if reason is None:
        reason = text.strip()

    # Extract actual PASS/FAIL from the reason text
    reason_upper = reason.upper()
    actual_from_reason = None
    if "PASS" in reason_upper and "FAIL" not in reason_upper:
        actual_from_reason = "PASS"
    elif "FAIL" in reason_upper and "PASS" not in reason_upper:
        actual_from_reason = "FAIL"
    
    # Use the parsed decision if we have it, otherwise use what we extracted from reason
    actual = decision if decision != "UNKNOWN" else actual_from_reason

    # Determine if there's a mismatch
    is_mismatch = (expected is not None) and (actual is not None) and (expected != actual)
    
    rows.append({
        "image": p.name,
        "expected": expected,
        "actual": actual,
        "reason": reason,
        "confidence": confidence,
        "is_mismatch": is_mismatch,
    })

results_df = pd.DataFrame(rows)

# Show only mismatches
mismatches_df = results_df[results_df['is_mismatch'] == True]
print(f"\nFound {len(mismatches_df)} mismatches:")
display(mismatches_df)

# Show summary
print(f"\nSummary:")
print(f"Total images: {len(results_df)}")
print(f"Mismatches: {len(mismatches_df)}")
print(f"Match rate: {((len(results_df) - len(mismatches_df)) / len(results_df) * 100):.1f}%")


Found 31 images
Processing Calgary_v6_model_should_fail.png...
Raw response: FAIL - The image only shows a partial view of the e-scooter (primarily the stem and part of the deck), not the entire scooter as required. Cannot properly assess if the scooter is parked correctly without seeing the full vehicle, including both wheels and the complete positioning relative to its surroundings.
Processing Calgary_v6_model_should_fail_2.jpg...
Raw response: FAIL - The e-scooter is parked on a sidewalk in a way that obstructs pedestrian traffic. The scooter is positioned across the middle of the walkway, forcing pedestrians to walk around it. There are no designated parking markings visible in the area, and the scooter placement creates an unnecessary obstacle on what appears to be a busy commercial sidewalk. The scooter should be parked parallel to the walkway or in a less obstructive position that leaves clear passage for pedestrians.
Processing calagry_sidewalk_should_pass_9.jpg...
Raw response

KeyboardInterrupt: 